In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
from numba import jit
inicio = time.time()

In [2]:
def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 200 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

In [3]:
dados = {}
objetivos = {}
variaveis = {}
tempos = {}
dados = extract_data()
lista_hs = [0.8, 0.6, 0.4, 0.2]

h=0.8
conjunto = 200
problema = 2
z_corte = 2

pi = []
ai = []
bi = []

pi = np.array(dados[conjunto][problema]['pi'])
ai = np.array(dados[conjunto][problema]['ai'])
bi = np.array(dados[conjunto][problema]['bi'])

d=sum(pi)*h

i_itens = len(pi)
i_index=list(range(i_itens))
i_np = np.array(i_index)

Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [4]:
sa = tuple([True,False]*int(i_itens*0.5)) #solucao atual

In [5]:
def transforma(solucao_partida):
    set_E = np.nonzero(solucao_partida == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao_partida == False)[0]
    return set_E,set_T



@jit(nopython=True)
def calcula_objetivo(set_E,set_T):
    if np.sum(pi[set_E])>d:

        return 1e15 #retorna valor imenso.

    ai_pi = ai[set_E]/pi[set_E] #apenas do set_E
    bi_pi = bi[set_T]/pi[set_T] #apenas do set_T
    set_E_pi = pi[set_E]
    set_T_pi = pi[set_T]
    ai_pi_decr = np.flip(np.argsort(ai_pi)) #ordem de ai_pi e depois por pi.
    bi_pi_decr = np.flip(np.argsort(bi_pi))
    set_E = set_E[ai_pi_decr]
    set_T = set_T[bi_pi_decr]
    
    
    sum_pi = 0
    objetivo = 0
    for tarefa in set_E:
        objetivo += sum_pi * ai[tarefa]
        sum_pi += pi[tarefa]
    
    sum_pi = 0
    for tarefa in set_T:
        objetivo += (sum_pi+pi[tarefa])*bi[tarefa]
        sum_pi += pi[tarefa]


    
    return objetivo#,True
                           
@jit(nopython=True)
def busca_total(solucao_partida): 
    objetivos = []
    for cada_serv in range(len(solucao_partida)):
        sol_em_teste = np.array(solucao_partida)
        sol_em_teste[cada_serv] = not(solucao_partida[cada_serv]) #inverte o serviço de bolso.
        set_E = np.nonzero(sol_em_teste == True)[0] #[0] para retornar o np array e não o tuple.
        set_T = np.nonzero(sol_em_teste == False)[0]
        objetivos.append(calcula_objetivo(set_E,set_T))
    return objetivos

In [6]:
%time objs = busca_total(sa)

Wall time: 2.97 s


In [7]:
%time objs = busca_total(sa)

Wall time: 3.03 ms


In [8]:
np.average(objs)

452966.44

In [9]:
objs

[449139.0,
 445683.0,
 449550.0,
 451524.0,
 454367.0,
 452946.0,
 454100.0,
 451172.0,
 452283.0,
 452616.0,
 454544.0,
 453327.0,
 458993.0,
 450642.0,
 456622.0,
 450252.0,
 458800.0,
 453160.0,
 446185.0,
 450203.0,
 455107.0,
 451684.0,
 457202.0,
 458665.0,
 456960.0,
 452515.0,
 455101.0,
 445308.0,
 457119.0,
 451038.0,
 454192.0,
 452757.0,
 454225.0,
 452561.0,
 454122.0,
 449478.0,
 454630.0,
 451301.0,
 451512.0,
 450039.0,
 454211.0,
 450672.0,
 455852.0,
 444257.0,
 455620.0,
 446904.0,
 459715.0,
 451603.0,
 458393.0,
 454377.0,
 453296.0,
 452937.0,
 447181.0,
 453405.0,
 456967.0,
 453151.0,
 461341.0,
 449666.0,
 453584.0,
 449755.0,
 454780.0,
 451464.0,
 458788.0,
 455849.0,
 450379.0,
 452194.0,
 456416.0,
 452959.0,
 455419.0,
 446650.0,
 449351.0,
 450097.0,
 453189.0,
 454453.0,
 456376.0,
 446743.0,
 453424.0,
 451627.0,
 451147.0,
 451973.0,
 452167.0,
 452786.0,
 454050.0,
 450749.0,
 458746.0,
 452494.0,
 453533.0,
 450121.0,
 454295.0,
 458885.0,
 446632.0,